In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

In [2]:
data = pd.read_csv('../data/filtered_reviews_in_Phonex.csv')
data.head(3)

,Unnamed: 0,review_id,user_id,business_id,stars,business_name,city,state,categories
0,1,aBWKb49Xfkv1946YN7_SIg,sSPbLBHcEMXaJfoO8zs1bA,poSV39UqEg-gpESXafS9-g,5.0,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."
1,2,jCHaWXgppmZjkOdpFltWGA,D5ywfFmwtJxLReqAYlDDmw,poSV39UqEg-gpESXafS9-g,5.0,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."
2,3,KvJ8yT-dODmCYe21J-Pp_A,gjoN4q-f61kwjmmU4mm1_g,poSV39UqEg-gpESXafS9-g,2.0,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."


### Construct Rating Matrix

In [3]:
def construct_rating_matrix(data, user, busi, rating):
    '''
    construct rating matrix from data
    the columns of which represent business_id
    the rows of which represent user_id
    the values of whose elements represent the according ratings
    '''
    user_id = data[user].unique().tolist()
    busi_id = data[busi].unique().tolist()
    sparse_matrix = np.zeros(shape=(len(user_id), len(busi_id)))
    
    ranking_df = data[[user, busi, rating]].groupby([user, busi]).count()
    ranking_df = ranking_df.reset_index()

    for i in range(len(ranking_df)):
        row_index, column_index = user_id.index(ranking_df.iloc[i, :].user_id), busi_id.index(ranking_df.iloc[i, :].business_id)
        element = ranking_df.iloc[i, :].stars
        sparse_matrix[row_index, column_index] = element
        
    # calculate sparstiy of the matrix
    sparsity = np.count_nonzero(sparse_matrix)/ (sparse_matrix.shape[0] * sparse_matrix.shape[1])
    
    return sparse_matrix, sparsity

In [4]:
sparse_matrix, sparsity = construct_rating_matrix(data, 'user_id', 'business_id', 'stars')

In [5]:
sparsity # 0.8%

0.008936948680351574

### Construct CF using SGD, with bias correction